In [1]:
import numpy as np
import keras 
from keras import layers
import tensorflow as tf
from keras.losses import Loss

2024-10-30 19:57:23.086959: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-30 19:57:23.099668: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1730318243.114942  137106 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1730318243.119004  137106 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-30 19:57:23.135092: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# Load the data and split it between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [3]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

2024-10-30 19:57:26.168577: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
I0000 00:00:1730318246.168692  137106 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1753 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Ti Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │        16,010 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 34,826 (136.04 KB)

 Trainable params: 34,826 (136.04 KB)

 Non-trainable params: 0 (0.00 B)

In [4]:
metric = keras.metrics.Accuracy(name="accuracy", dtype=None)
class CustomLoss(Loss):
    def call(self, y_true, y_pred):
        metric.update_state(tf.math.argmax(y_true, -1), tf.math.argmax(y_pred, -1))
        score = metric.result()
        # loss = -tf.reduce_sum(tf.one_hot(tf.math.argmax(y_pred, -1), 10) * tf.math.log(y_pred))
        loss = -tf.reduce_sum(y_true * tf.math.log(y_pred))
        return loss * score * 10
    
# a = np.array([[1,2,3,4,5,6,7,8,19,10], [1,2,3,4,5,6,7,8,9,10]], dtype=np.float32)
# b = np.array([[0,0,0,0,0,0,0,0,0,1], [0,0,0,0,0,0,0,0,0,1]])
# q = CustomLoss()
# print(tf.one_hot(tf.math.argmax(a, -1), 10))
# print(-tf.reduce_sum(b * tf.math.log(tf.nn.softmax(a, -1))))
# print(q(b,tf.nn.softmax(a, -1)))

In [5]:
# # pretrain
# batch_size = 1024
# epochs = 1
# model.compile(loss="categorical_crossentropy", optimizer="adamw", metrics=["accuracy"])
# model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

In [6]:
# alignment
batch_size = 16
epochs = 10
model.compile(loss=CustomLoss(), optimizer="adamw", metrics=["accuracy"])
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/10


I0000 00:00:1730318248.084620  137248 service.cc:148] XLA service 0x7fa8c0004c30 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1730318248.084695  137248 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Ti Laptop GPU, Compute Capability 8.6
2024-10-30 19:57:28.107738: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1730318248.213048  137248 cuda_dnn.cc:529] Loaded cuDNN version 90501


  61/3375 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.2831 - loss: 62.2542

I0000 00:00:1730318250.781546  137248 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


3375/3375 ━━━━━━━━━━━━━━━━━━━━ 14s 3ms/step - accuracy: 0.8822 - loss: 32.4699 - val_accuracy: 0.9862 - val_loss: 7.7306
Epoch 2/10
3375/3375 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.9757 - loss: 12.0309 - val_accuracy: 0.9888 - val_loss: 6.3550
Epoch 3/10
3375/3375 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - accuracy: 0.9812 - loss: 9.2946 - val_accuracy: 0.9898 - val_loss: 5.5375
Epoch 4/10
3375/3375 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.9822 - loss: 8.2678 - val_accuracy: 0.9910 - val_loss: 5.3779
Epoch 5/10
3375/3375 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.9850 - loss: 7.1153 - val_accuracy: 0.9907 - val_loss: 5.1124
Epoch 6/10
3375/3375 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.9865 - loss: 6.3443 - val_accuracy: 0.9888 - val_loss: 5.3838
Epoch 7/10
3375/3375 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.9877 - loss: 5.7868 - val_accuracy: 0.9915 - val_loss: 4.9846
Epoch 8/10
3375/3375 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.9879 - loss: 6.0247 - va